Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    Наименование вакансии.
    Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.
    
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
# Импортируем нужные для работы библиотеки и функции:

from bs4 import BeautifulSoup
import requests
import re
import time
from re import sub
from decimal import Decimal
import io
from datetime import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Основа для предстоящей работы: основная ссылка и юзер-агент, чтобы не заблочили. 
# Пользовательский ввод:
user_requests = input('Введите запрос: ').strip() # Мой запрос Python
total_pages = int(input('Введите колличество страниц для сбора данных: ')) # Запрашиваю 40 стр.

url = 'https://krasnoznamensk.hh.ru/search/vacancy?text=' + user_requests+ '&salary=&ored_clusters=true&enable_snippets=true&page=0'

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',}

map = {} # Создаю пустой словарь и счетчик 
id = 0 

start = time.time() # Функция для оценки времени

pause = time.sleep(5) # Функция для пауз

# Цикл для скрейпинга страниц
for page in range(total_pages):
    
    request_start_page = url + str(page)

    print("Scraping page: %d" % (page +1))

    # Мой основной запрос на скрейпинг всей страницы:
    my_requests = requests.get(request_start_page, headers=headers).text
    soup = BeautifulSoup(my_requests, 'html.parser')
    pause
    job_vacancy = soup.find_all("div", {"class": "vacancy-serp-item__layout"}) 
    
    print(len(job_vacancy))
    
     # Предохранитель(если что то не так с запросом, чтобы не случился "вечный двигатель"):

    page_nav = soup.find_all('a', {"class": "bloko-button"})

    if(len(page_nav) == 0):
        print("max page number: %d" % (page))
        end = time.time()
        print(end - start)
        break

    for vacancy in range(len(job_vacancy)):
        
        job = job_vacancy[vacancy]
        
        id += 1
        map[id] = {}

        #  Должность
        job_title = job.find('a', {"class": "serp-item__title"}).text
        

        # Зарплата (обрабатываем пропуски, через регулярные выражения обрабатываем формат и тут же разносим данные по разным колонкам)
        if job.find('span', {"class": "bloko-header-section-3"}) is None:
            salary = "Договорная"
            min_salary = "Не указанн"
            max_salary = "Не указанн"
            currency = 'Не указанно'
        else:
            salary = job.find('span', {"class": "bloko-header-section-3"}).text
            money = re.findall('[0-9]+\s?[0-9]+\s?[0-9]+', salary)
            currency = str(re.findall('\D+[^о]$', salary))
            for i in range(len(money)):
                if i == 0 :
                    min_salary = str(money[i]).replace('\u202f', '')
                elif i == 1:
                    max_salary = str(money[i]).replace('\u202f', '')
                    


        # Компания
        if job.find('a', {"class": "bloko-link bloko-link_kind-tertiary"}) is None:
            company = "Компания не указанна"
        else:
            company = job.find('a', {"class": "bloko-link bloko-link_kind-tertiary"}).text

        # Город
        if job.find('div',{"data-qa": "vacancy-serp__vacancy-address", "class": "bloko-text"}) is None:
            city = "Город не указанн"
        else:
            city = job.find('div',{"data-qa": "vacancy-serp__vacancy-address", "class": "bloko-text"}).text
        #city = job.find('div',{"data-qa": "vacancy-serp__vacancy-address", "class": "bloko-text"}).text
        
        # Описание вакансии(сразу обрабатываем возможные пропуски)
        if job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_responsibility", "class": "bloko-text"}) is None:
            vacancy_description = "Описание отсутствует"
        else:
            vacancy_description = job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_responsibility", "class": "bloko-text"}).text
        
        #Требования к соискателю (сразу обрабатываем возможные пропуски)
        if job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_requirement", "class": "bloko-text"}) is None:
            requirements_for_the_applicant = "Требования к соискателю не указанны"
        else:
            requirements_for_the_applicant = job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_requirement", "class": "bloko-text"}).text
        
        #Записываем все найденное в словарь:
        
        map[id]["Должность"] = job_title
        map[id]["Минимальная заработная плата"] = min_salary
        map[id]["Максимальная заработная плата"] = max_salary
        map[id]["Валюта"] = currency
        map[id]["Компания"] = company
        map[id]["Город"] = city
        map[id]["Описание вакансии"] = vacancy_description
        map[id]["Требования к соискателю"] = requirements_for_the_applicant
        
        

print(map)



Введите запрос: Python
Введите колличество страниц для сбора данных: 40
Scraping page: 1
20
Scraping page: 2
20
Scraping page: 3
20
Scraping page: 4
20
Scraping page: 5
20
Scraping page: 6
20
Scraping page: 7
20
Scraping page: 8
20
Scraping page: 9
20
Scraping page: 10
20
Scraping page: 11
20
Scraping page: 12
20
Scraping page: 13
20
Scraping page: 14
20
Scraping page: 15
20
Scraping page: 16
20
Scraping page: 17
20
Scraping page: 18
20
Scraping page: 19
20
Scraping page: 20
20
Scraping page: 21
20
Scraping page: 22
20
Scraping page: 23
20
Scraping page: 24
20
Scraping page: 25
20
Scraping page: 26
20
Scraping page: 27
20
Scraping page: 28
20
Scraping page: 29
20
Scraping page: 30
20
Scraping page: 31
20
Scraping page: 32
20
Scraping page: 33
20
Scraping page: 34
20
Scraping page: 35
20
Scraping page: 36
20
Scraping page: 37
20
Scraping page: 38
20
Scraping page: 39
20
Scraping page: 40
20
{1: {'Должность': 'Ведущий инженер по тестированию (python)', 'Минимальная заработная плата': 'Не

In [3]:
# Создаем df для собранных данных
hh_job_df = pd.DataFrame(columns=["Должность","Компания","Город","Минимальная заработная плата","Максимальная заработная плата",
                                  "Валюта","Описание вакансии","Требования к соискателю"])

In [4]:
# Заполняем df
for id in map.keys():
    df_dictionary = pd.DataFrame([map[id]])
    hh_job_df = pd.concat([hh_job_df, df_dictionary], ignore_index=True)

In [5]:
hh_job_df 


,Должность,Компания,Город,Минимальная заработная плата,Максимальная заработная плата,Валюта,Описание вакансии,Требования к соискателю
0,Ведущий инженер по тестированию (python),ЦРТ | Группа компаний,"Санкт-Петербург, Выборгская и еще 2",Не указанн,Не указанн,Не указанно,Разработка новых и поддержание в актуальном со...,Опыт в автоматизации тестирования от 2-3 лет. ...
1,Backend программист (Python),MANGO FZCO,Москва,120000,Не указанн,[' руб.'],Компания MANGO - разработка мобильного приложе...,Знание FastAPI. Асинхронность + многопоточност...
2,Младший инженер по тестированию (QA performanc...,ЦРТ | Группа компаний,"Санкт-Петербург, Выборгская и еще 2",Не указанн,Не указанн,Не указанно,Проводить анализ результатов нагрузочного тест...,"С python чуть сложнее, но тоже вполне реально)..."
3,Аналитик данных / Data Analyst,Сбер. Экспертам и руководителям,Москва,Не указанн,Не указанн,Не указанно,Проверять продуктовые гипотезы и готовить Ad-h...,Опыт работы с большими массивами данных. Знани...
4,Преподаватель Backend направления,ООО NEXTGEN ACADEMY,Ташкент,6000000,10000000,[' сум'],Проведение тренингов и обучающих курсов (Pytho...,Теоретические знания и практический опыт разра...
...,...,...,...,...,...,...,...,...
795,Инженер данных (Frontend),ООО Цифромед,"Москва, Деловой центр и еще 2",Не указанн,Не указанн,Не указанно,Проектирование и разработка BI и визуализации ...,"Опыт проектной деятельности по Agile (SCRUM, K..."
796,Devops инженер,Rubius,Москва,Не указанн,Не указанн,Не указанно,Развёртывание вспомогательного ПО в рамках Dev...,Умение работать самостоятельно. Внимательность...
797,Системный администратор Linux,ООО Комфортел,"Санкт-Петербург, Беговая и еще 1",100000,160000,[' руб.'],Поддерживать работоспособность всей серверной ...,Опыта организации мониторинга и работы с Zabbi...
798,DevOps инженер,ДИВАН.РУ,Владимир,Не указанн,Не указанн,Не указанно,Поддержание инфраструктуры dev/test/stage/prod...,Умение диагностировать и отлаживать проблемы п...


In [7]:
# Сохраняем получившийся df для дальнейшей работы 
hh_job_df.to_csv('hh_job_df.csv')